## scrape spreads 
http://www.footballlocks.com/nfl_point_spreads.shtml

In [1]:
import os
import pandas
import urllib2
import requests
import bs4

In [2]:
url = 'http://www.footballlocks.com/nfl_point_spreads.shtml'
r  = requests.get(url)
data = r.text

soup = bs4.BeautifulSoup(data, "lxml")

# get the tables w/ the spreads
tt = soup.findAll("table", {"width" : "580"})

In [37]:
df_spreads = pandas.DataFrame()
for i in range(2):  # this probably needs to change once the preseason is over
    dfs = pandas.read_html(str(tt[i]), )
    df_spreads = df_spreads.append(dfs[0])

df_spreads.index = range(len(df_spreads))
df_spreads.columns = ['date', 'favorite', 'spread', 'underdog']    
filter_bad = df_spreads.favorite == 'Favorite'

df_spreads = df_spreads[~filter_bad] 

# get the home favorite
df_spreads['home_favorite'] = df_spreads.favorite.str.contains('^At ')
# fix any spreads that are tied (PK)
df_spreads.loc[df_spreads.spread.astype(unicode).str.contains('PK'), 'spread'] = .1

# flip sign on spread for away favorite
df_spreads['factor'] = 1
df_spreads.loc[df_spreads.home_favorite == True,'factor'] = -1
df_spreads['spreads2'] = df_spreads.spread.astype(float) * df_spreads.factor

# get the home team
df_spreads['home_team'] = df_spreads.favorite
home_filter = df_spreads.underdog.str.contains('^At ') | df_spreads.underdog.str.contains('\(at ')
df_spreads.loc[home_filter, 'home_team'] = df_spreads.loc[home_filter, 'underdog']
df_spreads.home_team = df_spreads.home_team.str.replace('^At ', '')
df_spreads.home_team = df_spreads.home_team.str.replace('\(at .*\)', '')


df_spreads['datetime'] = pandas.to_datetime('2018/'+df_spreads.date.str.split(" ", expand=True)[0], format='%Y/%m/%d').dt.date
df_spreads

,date,favorite,spread,underdog,home_favorite,factor,spreads2,home_team,datetime
1,9/6 8:20 ET,At Philadelphia,-2.5,Atlanta,True,-1,2.5,Philadelphia,2018-09-06
2,9/9 1:00 ET,Pittsburgh,-5.5,At Cleveland,False,1,-5.5,Cleveland,2018-09-09
3,9/9 1:00 ET,At Minnesota,-6,San Francisco,True,-1,6.0,Minnesota,2018-09-09
4,9/9 1:00 ET,At Indianapolis,-3,Cincinnati,True,-1,3.0,Indianapolis,2018-09-09
5,9/9 1:00 ET,At Baltimore,-7,Buffalo,True,-1,7.0,Baltimore,2018-09-09
6,9/9 1:00 ET,Jacksonville,-3,At NY Giants,False,1,-3.0,NY Giants,2018-09-09
7,9/9 1:00 ET,At New Orleans,-9.5,Tampa Bay,True,-1,9.5,New Orleans,2018-09-09
8,9/9 1:00 ET,At New England,-6.5,Houston,True,-1,6.5,New England,2018-09-09
9,9/9 1:00 ET,Tennessee,-2,At Miami,False,1,-2.0,Miami,2018-09-09
10,9/9 4:05 ET,At LA Chargers,-3,Kansas City,True,-1,3.0,LA Chargers,2018-09-09


In [46]:
df_spreads.home_team.str.replace('\(at .*\)', '')


1     Philadelphia
2      Los Angeles
3        Minnesota
4     Indianapolis
5        Baltimore
6        NY Giants
7      New Orleans
8      New England
9            Miami
10     LA Chargers
11          Denver
12        Carolina
13         Arizona
14       Green Bay
15         Detroit
16         Oakland
Name: home_team, dtype: object

In [47]:
# stupid fix for London game
df_spreads.home_team[2] = 'Los Angeles'

/Users/amitbhattacharyya/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [48]:
# read in the master lines file
import datetime
codeDir = "".join([os.environ['MLNFL_ROOT'], os.path.sep])
path_to_lines = codeDir + "data/lines/"
lines_file = path_to_lines + "nflAllLines.csv"

df_lines = pandas.read_csv(lines_file)
df_lines.Date = pandas.to_datetime(df_lines.Date).dt.date
df_lines

,Date,Visitor,Visitor Score,Home Team,Home Score,Line,Total Line,season,week
0,1978-09-02,New York Giants,19.0,Tampa Bay Buccaneers,13.0,2.0,,1978,NaN
1,1978-09-03,Green Bay Packers,13.0,Detroit Lions,7.0,8.0,,1978,NaN
2,1978-09-03,Houston Oilers,14.0,Atlanta Falcons,20.0,-3.5,,1978,NaN
3,1978-09-03,Kansas City Chiefs,24.0,Cincinnati Bengals,23.0,8.0,,1978,NaN
4,1978-09-03,Miami Dolphins,20.0,New York Jets,33.0,-4.0,,1978,NaN
5,1978-09-03,Oakland Raiders,6.0,Denver Broncos,14.0,-4.0,,1978,NaN
6,1978-09-03,Pittsburgh Steelers,28.0,Buffalo Bills,17.0,-10.5,,1978,NaN
7,1978-09-03,Los Angeles Rams,16.0,Philadelphia Eagles,14.0,-6.0,,1978,NaN
8,1978-09-03,Los Angeles Chargers,24.0,Seattle Seahawks,20.0,-6.0,,1978,NaN
9,1978-09-03,San Francisco 49ers,7.0,Cleveland Browns,24.0,4.5,,1978,NaN


In [49]:
df_spreads.datetime.max()

datetime.date(2018, 9, 10)

In [51]:
# find the right week/game and update the spread
week_filter = (df_lines.Date <= df_spreads.datetime.max()) & (df_lines.Date >= df_spreads.datetime.min())

for ii,rr in df_spreads.iterrows():
    print ii, rr['home_team'], rr['spreads2']
    if 'NY' in rr['home_team']:
        rr['home_team'] = rr['home_team'].split(' ')[1]
    game_filter = df_lines[week_filter]['Home Team'].str.contains(rr['home_team'])
    irow = df_lines[week_filter][game_filter].index[0]
    df_lines.loc[irow, 'Line'] = rr['spreads2']

1 Philadelphia 2.5
2 Los Angeles -5.5
3 Minnesota 6.0
4 Indianapolis 3.0
5 Baltimore 7.0
6 NY Giants -3.0
7 New Orleans 9.5
8 New England 6.5
9 Miami -2.0
10 LA Chargers 3.0


IndexError: index 0 is out of bounds for axis 0 with size 0

> /Users/amitbhattacharyya/.virtualenvs/datascience/lib/python2.7/site-packages/pandas/core/indexes/base.py(1743)__getitem__()
   1741 
   1742         if is_scalar(key):
-> 1743             return getitem(key)
   1744 
   1745         if isinstance(key, slice):

ipdb> uo
*** NameError: name 'uo' is not defined
ipdb> up
> <ipython-input-51-31fb199cfefa>(9)<module>()
      6     if 'NY' in rr['home_team']:
      7         rr['home_team'] = rr['home_team'].split(' ')[1]
      8     game_filter = df_lines[week_filter]['Home Team'].str.contains(rr['home_team'])
----> 9     irow = df_lines[week_filter][game_filter].index[0]
     10     df_lines.loc[irow, 'Line'] = rr['spreads2']

ipdb> rr
date                9/9 4:05 ET
favorite         At LA Chargers
spread                       -3
underdog            Kansas City
home_favorite              True
factor                       -1
spreads2                      3
home_team           LA Chargers
datetime             2018-09-09
Name: 10, dtype: ob

In [14]:
lines_file = path_to_lines + "nflAllLines.csv"
df_lines.to_csv(lines_file, index=False)

In [44]:
pdb

Automatic pdb calling has been turned ON
